In [0]:
import zipfile
import os
import requests
import io

import pyspark.sql.functions as F
import pyspark.sql.types as T

# LINK to DATABRICKS - https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/1496180313079957/3631613044829155/1978268099723113/latest.html

In [0]:
url = "https://download.cms.gov/openpayments/PGYR2023_P01302025_01212025.zip"
local_filename = url.split("/")[-1] # Extracts the file name from the URL
buffer = io.BytesIO()
buffer.seek(0)

response = requests.get(url, stream=True)
# response = requests.get(url2, stream=True)

if response.status_code == 200:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            buffer.write(chunk)
    print(f"File downloaded successfully as {local_filename}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded successfully as PGYR2023_P01302025_01212025.zip


In [0]:
temp_dir = os.getcwd()
buffer.seek(0)

with zipfile.ZipFile(buffer, 'r') as zip_ref:
    # Loop through each file in the ZIP archive
    for file_name in zip_ref.namelist():
        table_name,ext = os.path.splitext(file_name)
        if ext.lower() == ".csv":
            temp_path = os.path.join(temp_dir, file_name)
            zip_ref.extract(file_name, temp_dir)
            df = spark.read.csv(f"file:{temp_path}", header=True, inferSchema=True)
            df.write.format("parquet").saveAsTable(table_name)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3631613044829181>:12
     10 zip_ref.extract(file_name, temp_dir)
     11 df = spark.read.csv(f"file:{temp_path}", header=True, inferSchema=True)
---> 12 df.write.format("parquet").saveAsTable(table_name)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1518 if format is not None:
   1519     self.format(format)
-> 1520 self._jwrite.saveAsTable(name)

File /dat

In [0]:
url2 = "https://download.cms.gov/openpayments/PHPRFL_P01302025_01212025.zip"
local_filename = url.split("/")[-1] # Extracts the file name from the URL
buffer = io.BytesIO()
buffer.seek(0)

response = requests.get(url2, stream=True)

if response.status_code == 200:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            buffer.write(chunk)
    print(f"File downloaded successfully as {local_filename}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

In [0]:
temp_dir = os.getcwd()
buffer.seek(0)

with zipfile.ZipFile(buffer, 'r') as zip_ref:
    # Loop through each file in the ZIP archive
    for file_name in zip_ref.namelist():
        table_name,ext = os.path.splitext(file_name)
        if ext.lower() == ".csv":
            temp_path = os.path.join(temp_dir, file_name)
            zip_ref.extract(file_name, temp_dir)
            df = spark.read.csv(f"file:{temp_path}", header=True, inferSchema=True)
            df.write.format("parquet").saveAsTable(table_name)

In [0]:
# df = spark.read.table(f"default.op_dtl_gnrl_pgyr2023_p01302025_01212025")
# df2 = spark.read.table(f"default.OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025")

PATH = "dbfs:/user/hive/warehouse/op_dtl_gnrl_pgyr2023_p01302025_01212025"
PATH2 = "dbfs:/user/hive/warehouse/op_cvrd_rcpnt_prfl_splmtl_p01302025_01212025"
df = spark.read.format('parquet').load(PATH)
df2 = spark.read.format('parquet').load(PATH2)

In [0]:
df.schema.fields

Out[21]: [StructField('Change_Type', StringType(), True),
 StructField('Covered_Recipient_Type', StringType(), True),
 StructField('Teaching_Hospital_CCN', StringType(), True),
 StructField('Teaching_Hospital_ID', StringType(), True),
 StructField('Teaching_Hospital_Name', StringType(), True),
 StructField('Covered_Recipient_Profile_ID', StringType(), True),
 StructField('Covered_Recipient_NPI', StringType(), True),
 StructField('Covered_Recipient_First_Name', StringType(), True),
 StructField('Covered_Recipient_Middle_Name', StringType(), True),
 StructField('Covered_Recipient_Last_Name', StringType(), True),
 StructField('Covered_Recipient_Name_Suffix', StringType(), True),
 StructField('Recipient_Primary_Business_Street_Address_Line1', StringType(), True),
 StructField('Recipient_Primary_Business_Street_Address_Line2', StringType(), True),
 StructField('Recipient_City', StringType(), True),
 StructField('Recipient_State', StringType(), True),
 StructField('Recipient_Zip_Code', Strin

In [0]:
df = df.withColumn("Total_Amount_of_Payment_USDollars", F.col("Total_Amount_of_Payment_USDollars").cast(T.FloatType()))

filtered_df = df.filter(F.col("Total_Amount_of_Payment_USDollars") > 1000)

g_df = filtered_df.groupBy([F.col("Nature_of_Payment_or_Transfer_of_Value")]).agg(
    F.count(F.col("Total_Amount_of_Payment_USDollars")).alias("count"),
    F.sum(F.col("Total_Amount_of_Payment_USDollars")).alias("total_amount_sum")
)

In [0]:
g_df = g_df.orderBy("count", ascending  = False)
g_df.show()

+--------------------------------------+------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value| count|    total_amount_sum|
+--------------------------------------+------+--------------------+
|                  Compensation for ...|164093| 5.582959709562378E8|
|                        Consulting Fee|105239| 4.819268881451416E8|
|                    Travel and Lodging| 24793| 5.476596304553223E7|
|                             Honoraria| 13750| 5.311015340332031E7|
|                             Education| 13376| 5.179660454785156E7|
|                    Royalty or License| 11538| 1.190650138420227E9|
|                  Compensation for ...|  8658| 2.762957946057129E7|
|                                 Grant|  4922|1.1127899889367676E8|
|                  Space rental or f...|  4917| 2.768891679724121E7|
|                  Long term medical...|  2930|2.9007746673583984E7|
|                      Debt forgiveness|  1788|1.3169610816955566E7|
|                     Food and Bev

In [0]:
g_df.limit(10).show(truncate = False)

+--------------------------------------+------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value| count|    total_amount_sum|
+--------------------------------------+------+--------------------+
|                  Compensation for ...|164093| 5.582959709562378E8|
|                        Consulting Fee|105239| 4.819268881451416E8|
|                    Travel and Lodging| 24793| 5.476596304553223E7|
|                             Honoraria| 13750| 5.311015340332031E7|
|                             Education| 13376| 5.179660454785156E7|
|                    Royalty or License| 11538| 1.190650138420227E9|
|                  Compensation for ...|  8658| 2.762957946057129E7|
|                                 Grant|  4922|1.1127899889367676E8|
|                  Space rental or f...|  4917| 2.768891679724121E7|
|                  Long term medical...|  2930|2.9007746673583984E7|
+--------------------------------------+------+--------------------+



In [0]:
g_df = g_df.orderBy("total_amount_sum", ascending  = False)
g_df.limit(10).show(truncate = False)

+--------------------------------------+------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value| count|    total_amount_sum|
+--------------------------------------+------+--------------------+
|                    Royalty or License| 11538| 1.190650138420227E9|
|                  Compensation for ...|164093| 5.582959709562378E8|
|                        Consulting Fee|105239| 4.819268881451416E8|
|                                 Grant|  4922|1.1127899889367676E8|
|                          Acquisitions|   563| 7.191250424938965E7|
|                    Travel and Lodging| 24793| 5.476596304553223E7|
|                             Honoraria| 13750| 5.311015340332031E7|
|                             Education| 13376| 5.179660454785156E7|
|                  Long term medical...|  2930|2.9007746673583984E7|
|                  Space rental or f...|  4917| 2.768891679724121E7|
+--------------------------------------+------+--------------------+



In [0]:
g_df2 = df.groupBy([F.col("Covered_Recipient_Specialty_1")]).agg(
    F.count(F.col("Total_Amount_of_Payment_USDollars")).alias("count"),
    F.sum(F.col("Total_Amount_of_Payment_USDollars")).alias("total_amount_sum")
)

In [0]:
g_df2 = g_df2.orderBy("total_amount_sum", ascending  = False)
g_df2.limit(10).show(truncate = False)

+------------------------------------------------------------------------------------------------+-------+--------------------+
|Covered_Recipient_Specialty_1                                                                   |count  |total_amount_sum    |
+------------------------------------------------------------------------------------------------+-------+--------------------+
|null                                                                                            |30686  |7.936674627555835E8 |
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                         |210504 |4.0345021277223873E8|
|Allopathic & Osteopathic Physicians|Internal Medicine                                           |1307850|1.3136300312194332E8|
|Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology                            |457774 |8.979213624269351E7 |
|Allopathic & Osteopathic Physicians|Neurological Surgery                                        |78680 

In [0]:
g_df3 = df.groupBy([F.col("Covered_Recipient_Primary_Type_1")]).agg(
    F.count(F.col("Total_Amount_of_Payment_USDollars")).alias("count"),
    F.sum(F.col("Total_Amount_of_Payment_USDollars")).alias("total_amount_sum")
)


In [0]:
g_df3 = g_df3.orderBy("total_amount_sum", ascending  = False)
g_df3.limit(10).show(truncate = False)

+--------------------------------------+-------+--------------------+
|Covered_Recipient_Primary_Type_1      |count  |total_amount_sum    |
+--------------------------------------+-------+--------------------+
|Medical Doctor                        |7913524|2.0401834645222669E9|
|null                                  |30588  |7.933900793164766E8 |
|Nurse Practitioner                    |3266415|1.3546701068595254E8|
|Doctor of Osteopathy                  |941641 |9.368468839218245E7 |
|Doctor of Dentistry                   |350066 |8.315500001068492E7 |
|Physician Assistant                   |1618627|6.882057343604396E7 |
|Doctor of Podiatric Medicine          |148617 |3.442229348336857E7 |
|Doctor of Optometry                   |243641 |2.331802892928546E7 |
|Certified Registered Nurse Anesthetist|47191  |2143071.420371622   |
|Clinical Nurse Specialist             |27756  |2023960.0999247283  |
+--------------------------------------+-------+--------------------+

